<a href="https://colab.research.google.com/github/DAEN690-S22-OA/OceanicAirspaceProject/blob/main/VerticalSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting Up The Environment

In [1]:
#Installing the required libraries
!pip install geopy
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=f5ddc8d28888ee3751af22f5bf467e8a09d50e2f006f4567f2af98991f476207
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [2]:
#Importing the required libraries
import pandas as pd
import pandasql as ps
# from pyspark.sql import SparkSession
import os
import sys
import re
from datetime import datetime
from geopy.distance import geodesic as GD
from math import radians, cos, sin, asin, sqrt
from google.colab import files
import io

# Importing Raw Data & Pre-Processing

In [ ]:
#Upload the full datafile 
uploadedfile = files.upload('DEC_24_2021.csv')
rawData_df = pd.read_csv(io.BytesIO(uploadedfile['DEC_24_2021.csv']))
print(f'Total record count : ',len(rawData_df.index))

In [ ]:
#New dataframe with selected attributes from the raw data (Flight Level Column added)
airspaceData_df =rawData_df[["FRN73TMRPDateTimeOfMessageRec","FRN131HRPWCFloatingPointLat","FRN131HRPWCFloatingPointLong","FRN145FLFlightLevel",
                 "FRN170TITargetId","RESHSelectedHeading","FRN80TATargetAddress",
                 "FRN161TNTrackNumber"]]

#Rename columns to make it easier to read
airspaceData_df = airspaceData_df.rename(columns={'FRN73TMRPDateTimeOfMessageRec': 'DateTime', 
                                                  'FRN131HRPWCFloatingPointLat': "Latitude", 
                                                  'FRN131HRPWCFloatingPointLong': "Longitude",  
                                                  'FRN145FLFlightLevel':"Height",
                                                  'FRN170TITargetId': "TargetID", 
                                                  'RESHSelectedHeading': "SelectedHeading", 
                                                  'FRN80TATargetAddress': "TargetAddress",
                                                  'FRN161TNTrackNumber': "TrackNumber"})

#Preview to ensure everything is looking as expected 
airspaceData_df.head()

In [ ]:
# Remove anything less than 240 flight level 
airspaceData1 = airspaceData_df[(airspaceData_df['Height'] >= 240)]

#Preview Data
airspaceData1.head()

In [ ]:
# Convert dask dataframe to pandas dataframe
airspaceDF = airspaceData1.compute()

In [ ]:
# Remove unnescessary characters 
char = ['T','Z']
for x in char:
    airspaceDF["DateTime"] = airspaceDF["DateTime"].str.replace( x ," ")

# Formatted Datetime
airspaceDF["DateTime"] = pd.to_datetime(airspaceDF["DateTime"], format="%Y-%m-%d %H:%M:%S")

In [ ]:
# Create 4 new columns for Hour, Minute, Second and Microsecond
airspaceDF["Hour"] = airspaceDF["DateTime"].dt.hour
airspaceDF["Minute"] = airspaceDF["DateTime"].dt.minute
airspaceDF["Second"] = airspaceDF["DateTime"].dt.second
airspaceDF["microSecond"] = airspaceDF["DateTime"].dt.microsecond

In [ ]:
# Reorder columns
airspaceDF = airspaceDF[["DateTime","Hour","Minute","Second","microSecond","Latitude","Longitude","Height",
                                   "TargetID","SelectedHeading","TargetAddress",
                                   "TrackNumber"]]

# Preview Dataframe
airspaceDF.head()

## Upload datafile here during testing

In [3]:
#---------------REMOVE AFTER FINAL TESTING --------------------------------

#Upload the data file after pre-processing (only the selected attribues and only above level 240)
uploadedfile = files.upload()
airspaceDF = pd.read_csv(io.BytesIO(uploadedfile['airspaceData.csv']))

Saving airspaceData.csv to airspaceData.csv


In [10]:
airspaceDF.head(5)

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 12:36:02.773,12,36,2,773000,37.307144,-139.574170,350.00,ASA899,NaN,A7B779,1024.0
1,2021-12-24 04:13:17.836,4,13,17,836000,21.496660,-146.918742,360.00,HAL50,NaN,A48E9E,1024.0
2,2021-12-24 23:59:59.984,23,59,59,984000,25.936523,-149.628348,380.00,SWA2385,45.703125,ABF949,1024.0
3,2021-12-24 20:54:11.117,20,54,11,117000,28.167389,-127.572661,390.00,FJI810,43.593750,C8809D,1024.0
4,2021-12-24 12:36:02.750,12,36,2,750000,48.981194,-131.626113,389.75,JAL10,70.312500,86E84C,624.0


In [11]:
#Some column names may be imported with leading spaces. This will remove that
airspaceDF.columns = [x.strip() for x in airspaceDF.columns]

print(airspaceDF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2540762 entries, 0 to 2540761
Data columns (total 12 columns):
 #   Column           Dtype  
---  ------           -----  
 0   DateTime         object 
 1   Hour             int64  
 2   Minute           int64  
 3   Second           int64  
 4   microSecond      int64  
 5   Latitude         float64
 6   Longitude        float64
 7   Height           float64
 8   TargetID         object 
 9   SelectedHeading  float64
 10  TargetAddress    object 
 11  TrackNumber      float64
dtypes: float64(5), int64(4), object(3)
memory usage: 232.6+ MB
None


# Filter A Time Block

In [22]:
def minuteFilter(HourCounter,MinuteCounter):

  global airspaceDF

  #create SQL query for flights between the start and end time
  sql1 = "SELECT *, min(Second) FROM airspaceDF WHERE Hour = '{0}' and Minute = '{1}' GROUP BY TargetID ORDER BY TargetID, Second".format(HourCounter, MinuteCounter)

  #Run query and store results
  recordsInMinute = ps.sqldf(sql1, globals())
  del recordsInMinute['min(Second)']

  recordCount = len(recordsInMinute.index)

  return (recordCount)

# Check For Proximity

In [ ]:
#Create tables to store proximity
sql3 = "SELECT * FROM firstEntryDF ORDER BY Latitude"
sql4 = "SELECT * FROM firstEntryDF ORDER BY Longitude"

#Run query and store results
LatitudeOrderDF = ps.sqldf(sql3, locals())
LongitudeOrderDF = ps.sqldf(sql4, locals())


In [ ]:
LatitudeOrderDF.head()

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,350.00,FDX77,210.937500,ABE1EB,1024.0
1,2021-12-24 23:59:00.625,23,59,0,625000,6.845460,-163.240356,339.75,MGE600,NaN,AC3C99,1484.0
2,2021-12-24 23:59:03.977,23,59,3,977000,11.171885,178.954757,350.00,ATN440,51.328125,AA21BF,1024.0
3,2021-12-24 23:59:01.453,23,59,1,453000,11.610489,-166.661035,400.00,HAL451,NaN,A45233,1024.0
4,2021-12-24 23:59:04.836,23,59,4,836000,12.478409,142.616035,380.00,SIA8890,258.750000,76B44A,1997.0


In [ ]:
LongitudeOrderDF.head()

,DateTime,Hour,Minute,Second,microSecond,Latitude,Longitude,Height,TargetID,SelectedHeading,TargetAddress,TrackNumber
0,2021-12-24 23:59:03.648,23,59,3,648000,17.969330,-176.990517,360.00,MGE300,NaN,A9228A,1088.0
1,2021-12-24 23:59:04.047,23,59,4,47000,0.190538,-175.769980,350.00,FDX77,210.937500,ABE1EB,1024.0
2,2021-12-24 23:59:02.883,23,59,2,883000,33.536041,-175.560315,400.00,UAL903,288.281250,AA9321,1488.0
3,2021-12-24 23:59:00.430,23,59,0,430000,49.627487,-174.973249,339.75,UAL893,265.078125,A31A73,1024.0
4,2021-12-24 23:59:00.109,23,59,0,109000,47.560684,-172.337723,400.00,AAL9726,277.031250,AB5257,1024.0


In [ ]:
# Implement the formula below
def distance_d(point0,pointX):
# The function "radians" is found in the math module, It's also used to convert radians to degrees.  
 LoA = radians(point0[1])  
 LoB = radians(pointX[1])
 LaA=  radians(point0[0])  
 LaB = radians(pointX[0]) 
# The "Haversine formula" is used.
 D_Lo = LoB - LoA 
 D_La = LaB - LaA 
 P = sin(D_La / 2)**2 + cos(LaA) * cos(LaB) * sin(D_Lo / 2)**2  
   
 Q = 2 * asin(sqrt(P))   
 # The earth's radius in kilometers.
 R_km = 6371  
 
 # Change the kilometer to  nautical miles
 R_nm = R_km*0.539956803

# Then we'll compute the outcome.
 return(Q * R_nm)
 


In [ ]:
#Calculate the distance of the points closest to each other by longitidue and latitude



longpoint_a = []
longpoint_b = []
longdistance_ab = []

latpoint_a = []
latpoint_b = []
latdistance_ab = []

for a in LongitudeOrderDF.index:   
    for n in range(1,len(LongitudeOrderDF)): 
    # check distance with next flights until found distance >25, then out of the inner loop
        b = a+n
        if b < len(LongitudeOrderDF):
            point0 = LongitudeOrderDF.loc[a,'Latitude'], LongitudeOrderDF.loc[a,'Longitude']
            pointX = LongitudeOrderDF.loc[b,'Latitude'], LongitudeOrderDF.loc[b,'Longitude']    
            distance = distance_d(point0,pointX) #***change distance calculation***
            if distance <= 25: #***change limit closet distance***
                longpoint_a.append(a)
                longpoint_b.append(b)
                longdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by longitiude","\n",longpoint_a,"\n",longpoint_b,"\n", longdistance_ab,"\n")

for a in LatitudeOrderDF.index:   
    for n in range(1,len(LatitudeOrderDF)): 
    # check distance with next flights until found distance > 5, then out of the inner loop
        b = a+n
        if b < len(LatitudeOrderDF):
            point0 = LatitudeOrderDF.loc[a,'Latitude'], LatitudeOrderDF.loc[a,'Longitude']
            pointX = LatitudeOrderDF.loc[b,'Latitude'], LatitudeOrderDF.loc[b,'Longitude']    
            distance = distance_d(point0,pointX)  #***change distance calculation***
            if distance <= 25: #***change limit closet distance***
                latpoint_a.append(a)
                latpoint_b.append(b)
                latdistance_ab.append(distance)
            else:
                break
print("Points closest to each other by latitiude","\n",latpoint_a,"\n", latpoint_b,"\n", latdistance_ab)

Points closest to each other by longitiude 
 [6, 14, 28, 47, 52, 52, 53, 56, 58, 63, 76, 98, 102, 107, 110, 111, 120, 138, 141] 
 [7, 15, 29, 48, 53, 54, 54, 57, 59, 64, 77, 99, 103, 108, 111, 112, 121, 139, 142] 
 [14.564466984444943, 4.352174689134579, 21.581335482043777, 22.37790018229146, 1.5363001001512926, 23.653987596900862, 22.118858754552214, 7.235764008897653, 5.014677098078101, 1.7272867504004672, 21.716415908416437, 24.68127001669846, 12.028230524620557, 9.413652830843938, 24.07829764607354, 23.84771435007522, 2.180264091774546, 10.397545648681234, 20.115670271210394] 

Points closest to each other by latitiude 
 [14, 14, 15, 28, 35, 48, 55, 57, 61, 66, 68, 70, 76, 83, 84, 109, 130, 132] 
 [15, 16, 16, 29, 36, 49, 56, 58, 62, 67, 69, 71, 77, 84, 85, 110, 131, 133] 
 [17.92096270841433, 11.695486593102506, 19.944749004866203, 7.235764008897653, 4.352174689134579, 9.413652830843938, 3.5440342000344183, 1.5363001001512926, 5.014677098078101, 14.867985730129725, 1.7272867504004

In [ ]:
# Create function to select, merge and add the values from analyzing Longitude and Latitude
def newDF(OrderDF,x,y,d):
    """DF is Long/LatitudeOrderDF
       x = long/latpoint_a
       y = long/latpoint_b
       d = long/latdistance_ab"""
    # select rows that index is in list 'point_a', 'point_b'
    A = OrderDF.loc[x,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    B = OrderDF.loc[y,['DateTime','Hour','Minute','Second','microSecond','Latitude','Longitude','Height',
                             'TargetID', 'SelectedHeading', 'TargetAddress']]
    # Join 2 tables by the "TargetID" of point a (for the uniquness)
    OrderResult = pd.merge(A.reset_index(drop=True),B.reset_index(drop=True),left_index=True, right_index=True)
    # add distance column
    OrderResult['Distance'] = d
    return OrderResult

In [ ]:
# Apply function to select and merge data frame
LongOrderResult = newDF(LongitudeOrderDF,longpoint_a, longpoint_b,longdistance_ab)
LatOrderResult = newDF(LatitudeOrderDF,latpoint_a, latpoint_b,latdistance_ab)

In [ ]:
# Concatenate results from order by longitude and latitude
Resultsdf = pd.concat([LongOrderResult,LatOrderResult]).reset_index(drop=True)

# Show result
Resultsdf

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:02.734,23,59,2,734000,28.675294,-169.513442,360.00,KAL054,296.015625,...,59,2,508000,28.527740,-169.294140,310.00,JAL783,296.718750,86220C,14.564467
1,2021-12-24 23:59:02.086,23,59,2,86000,24.062439,-163.081970,380.00,HAL459,NaN,...,59,2,773000,24.034607,-163.008677,400.00,JAL8781,295.312500,86D996,4.352175
2,2021-12-24 23:59:03.641,23,59,3,641000,23.053459,-155.782878,350.00,UAL2380,218.671875,...,59,1,977000,22.696078,-155.741119,428.00,N1898B,NaN,A1658D,21.581335
3,2021-12-24 23:59:01.461,23,59,1,461000,21.901062,-153.453036,400.25,HAL54,NaN,...,59,2,391000,21.528900,-153.431213,329.75,AAL58,0.000000,A4E307,22.377900
4,2021-12-24 23:59:01.352,23,59,1,352000,26.954584,-152.813592,320.00,ASA843,215.859375,...,59,0,969000,26.972443,-152.793032,400.00,N11HM,NaN,A02A95,1.536300
5,2021-12-24 23:59:01.352,23,59,1,352000,26.954584,-152.813592,320.00,ASA843,215.859375,...,59,2,617000,27.218348,-152.484900,340.00,ASA103,215.156250,A5A11A,23.653988
6,2021-12-24 23:59:00.969,23,59,0,969000,26.972443,-152.793032,400.00,N11HM,NaN,...,59,2,617000,27.218348,-152.484900,340.00,ASA103,215.156250,A5A11A,22.118859
7,2021-12-24 23:59:03.742,23,59,3,742000,23.259659,-152.178922,380.00,SWA1385,203.203125,...,59,3,555000,23.285057,-152.050680,340.00,SWA341,234.140625,AC08B1,7.235764
8,2021-12-24 23:59:03.719,23,59,3,719000,27.598029,-152.045975,350.00,ASA860,NaN,...,59,2,844000,27.656204,-151.978334,360.00,ASA879,215.156250,A6B7EC,5.014677
9,2021-12-24 23:59:03.859,23,59,3,859000,28.382806,-151.096640,310.25,ASA896,35.156250,...,59,1,898000,28.411063,-151.090501,330.00,ASA832,NaN,A78C43,1.727287


In [ ]:
# Delete duplicate pairs of TargetID x and y
ResultsDF = Resultsdf.drop_duplicates(subset = ['TargetID_x', 'TargetID_y','Distance'],keep = 'last').reset_index(drop = True)
ResultsDF

,DateTime_x,Hour_x,Minute_x,Second_x,microSecond_x,Latitude_x,Longitude_x,Height_x,TargetID_x,SelectedHeading_x,...,Minute_y,Second_y,microSecond_y,Latitude_y,Longitude_y,Height_y,TargetID_y,SelectedHeading_y,TargetAddress_y,Distance
0,2021-12-24 23:59:02.734,23,59,2,734000,28.675294,-169.513442,360.00,KAL054,296.015625,...,59,2,508000,28.527740,-169.294140,310.00,JAL783,296.718750,86220C,14.564467
1,2021-12-24 23:59:02.086,23,59,2,86000,24.062439,-163.081970,380.00,HAL459,NaN,...,59,2,773000,24.034607,-163.008677,400.00,JAL8781,295.312500,86D996,4.352175
2,2021-12-24 23:59:03.641,23,59,3,641000,23.053459,-155.782878,350.00,UAL2380,218.671875,...,59,1,977000,22.696078,-155.741119,428.00,N1898B,NaN,A1658D,21.581335
3,2021-12-24 23:59:01.461,23,59,1,461000,21.901062,-153.453036,400.25,HAL54,NaN,...,59,2,391000,21.528900,-153.431213,329.75,AAL58,0.000000,A4E307,22.377900
4,2021-12-24 23:59:01.352,23,59,1,352000,26.954584,-152.813592,320.00,ASA843,215.859375,...,59,2,617000,27.218348,-152.484900,340.00,ASA103,215.156250,A5A11A,23.653988
5,2021-12-24 23:59:00.969,23,59,0,969000,26.972443,-152.793032,400.00,N11HM,NaN,...,59,2,617000,27.218348,-152.484900,340.00,ASA103,215.156250,A5A11A,22.118859
6,2021-12-24 23:59:00.219,23,59,0,219000,26.948578,-149.704408,409.75,AAL692,227.812500,...,59,3,16000,27.139210,-149.359289,340.00,SWA808,229.218750,AB6247,21.716416
7,2021-12-24 23:59:02.750,23,59,2,750000,28.869736,-146.095927,390.00,SWA1822,225.703125,...,59,0,94000,29.076096,-145.689539,360.00,ASA149,NaN,A76FE9,24.681270
8,2021-12-24 23:59:02.695,23,59,2,695000,43.968775,-139.159960,330.00,SIA11,269.296875,...,59,2,570000,43.966690,-139.109589,380.00,JAL6721,267.890625,86E474,2.180264
9,2021-12-24 23:59:02.766,23,59,2,766000,30.994573,-132.502643,450.00,LXJ587,239.765625,...,59,0,273000,31.320099,-132.410027,360.00,ANZ1007,222.187500,C820CE,20.115670


# Calculate Height Differences

In [ ]:
heightDifference = []
potentialLoss1000 = []
potentialLoss400 = []

counter = 0

while counter < len(ResultsDF):
  difference = (abs((ResultsDF['Height_x'][counter]) - (ResultsDF['Height_y'][counter])))*10
  heightDifference.append(difference)

  if difference <= 1000:
    potentialLoss1000.append('True')
    if difference <= 400:
      potentialLoss400.append('True')
    else:
      potentialLoss400.append('False')
  else:
    potentialLoss1000.append('False')
    potentialLoss400.append('False')

  counter = counter + 1

print(heightDifference)
print(potentialLoss1000)
print(potentialLoss400)


[500.0, 200.0, 780.0, 705.0, 200.0, 600.0, 697.5, 300.0, 500.0, 900.0, 400.0, 102.5, 502.5, 400.0, 200.0, 100.0, 1097.5, 800.0, 100.0, 1000.0, 197.5, 500.0, 300.0, 200.0, 100.0, 400.0, 100.0, 500.0]
['True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True']
['False', 'True', 'False', 'False', 'True', 'False', 'False', 'True', 'False', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'False']


In [ ]:
ResultsDF['HeightDifference'] = heightDifference
ResultsDF['potentialLoss400'] = potentialLoss400
ResultsDF['potentialLoss1000'] = potentialLoss1000

print(ResultsDF)

                 DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0   2021-12-24 23:59:02.734      23        59         2         734000   
1   2021-12-24 23:59:02.086      23        59         2          86000   
2   2021-12-24 23:59:03.641      23        59         3         641000   
3   2021-12-24 23:59:01.461      23        59         1         461000   
4   2021-12-24 23:59:01.352      23        59         1         352000   
5   2021-12-24 23:59:00.969      23        59         0         969000   
6   2021-12-24 23:59:00.219      23        59         0         219000   
7   2021-12-24 23:59:02.750      23        59         2         750000   
8   2021-12-24 23:59:02.695      23        59         2         695000   
9   2021-12-24 23:59:02.766      23        59         2         766000   
10  2021-12-24 23:59:01.828      23        59         1         828000   
11  2021-12-24 23:59:01.828      23        59         1         828000   
12  2021-12-24 23:59:03.383      23   

# Call the functions

In [23]:
HourCounter = 0
MinuteCounter = 0

while MinuteCounter < 3:
  recordsByMinuteDF = minuteFilter(HourCounter,MinuteCounter)
  if MinuteCounter == 59:
    MinuteCounter = 0
    HourCounter = HourCounter + 1
  else:
    MinuteCounter = MinuteCounter + 1
  print(recordsByMinuteDF)

175
176
176


# Print Results

In [ ]:
ResultsDF.to_csv('results2359.csv', encoding = 'utf-8-sig') 
files.download('results2359.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Get the first entry for this minute of time
sql6 = "SELECT * FROM ResultsDF WHERE potentialLoss400 = 'True' "

#Run query and store results
LossCandidates400 = ps.sqldf(sql6, locals())

In [ ]:
print(LossCandidates400)

                 DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0   2021-12-24 23:59:02.086      23        59         2          86000   
1   2021-12-24 23:59:01.352      23        59         1         352000   
2   2021-12-24 23:59:02.750      23        59         2         750000   
3   2021-12-24 23:59:01.828      23        59         1         828000   
4   2021-12-24 23:59:01.828      23        59         1         828000   
5   2021-12-24 23:59:03.742      23        59         3         742000   
6   2021-12-24 23:59:02.773      23        59         2         773000   
7   2021-12-24 23:59:01.313      23        59         1         313000   
8   2021-12-24 23:59:03.719      23        59         3         719000   
9   2021-12-24 23:59:03.859      23        59         3         859000   
10  2021-12-24 23:59:00.836      23        59         0         836000   
11  2021-12-24 23:59:03.836      23        59         3         836000   
12  2021-12-24 23:59:04.477      23   

In [ ]:
#Get the first entry for this minute of time
sql7 = "SELECT * FROM ResultsDF WHERE potentialLoss1000 = 'True' "

#Run query and store results
LossCandidates1000 = ps.sqldf(sql7, locals())

In [ ]:
print(LossCandidates1000)

                 DateTime_x  Hour_x  Minute_x  Second_x  microSecond_x  \
0   2021-12-24 23:59:02.734      23        59         2         734000   
1   2021-12-24 23:59:02.086      23        59         2          86000   
2   2021-12-24 23:59:03.641      23        59         3         641000   
3   2021-12-24 23:59:01.461      23        59         1         461000   
4   2021-12-24 23:59:01.352      23        59         1         352000   
5   2021-12-24 23:59:00.969      23        59         0         969000   
6   2021-12-24 23:59:00.219      23        59         0         219000   
7   2021-12-24 23:59:02.750      23        59         2         750000   
8   2021-12-24 23:59:02.695      23        59         2         695000   
9   2021-12-24 23:59:02.766      23        59         2         766000   
10  2021-12-24 23:59:01.828      23        59         1         828000   
11  2021-12-24 23:59:01.828      23        59         1         828000   
12  2021-12-24 23:59:03.383      23   